This code sets up and trains a deep learning model to learn the parameters of a SIRD (Susceptible, Infected, Recovered, Deceased) model for the spread of a disease, like COVID-19, in a population.

The SIRD model is a compartmental model that represents the number of people in a population that are susceptible (S), infected (I), recovered (R), and deceased (D). It includes parameters for the rates at which individuals move from one compartment to another: 
- β is the transmission rate 
- γ is the recovery rate 
- δ is the mortality rate

The code does the following:

1. It imports necessary libraries and packages, including TensorFlow, NumPy, Pandas, Scikit-learn, Scipy, Matplotlib, and others.
2. It sets the directory paths and creates a new directory for storing figures, if it doesn't already exist.
3. It sets the seed for the random number generators in NumPy and TensorFlow to ensure the reproducibility of the results.
4. It defines a class named `SIRD` that represents the SIRD model. 

   - The `SIRD` class has methods for initializing the model (creating the neural network), defining the model's dynamics in `net_sird`, defining parameters in `net_param`, calculating residuals in `net_residual`, training the model, and making predictions.

5. It reads the dataset named 'tndata.csv' and preprocesses the data into the form that can be input into the SIRD model.

6. It creates an instance of the `SIRD` class and trains it.

7. After the model has been trained, it makes predictions for the parameters and compartments.

8. It calculates error metrics like RMSE (Root Mean Squared Error), MAPE (Mean Absolute Percentage Error), and EV (Explained Variance) to evaluate the performance of the model.

The SIRD model is implemented as a Physics-informed Neural Network (PINN), where the physical laws governing the disease spread (the SIRD equations) are used as a part of the loss function to guide the training of the model. This approach allows the model to learn not only from the data but also from the underlying physical laws, improving the generalization capability and interpretability of the model.

In [ ]:
import tensorflow.compat.v1 as tf  #Torku
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy import optimize
import matplotlib.dates as dates
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import explained_variance_score
# from ipywidgets import interact, widgets
from scipy.integrate import solve_ivp
plt.style.use('seaborn-poster')
matplotlib.rcParams['figure.figsize'] = (10., 6.)
import copy
# import sympy
# %matplotlib inline
import scipy as sp
from scipy.integrate import odeint
import datetime as dt
import timeit
import time
import os
import sys
import time
sys.path.insert(0, '../../Utilities/')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from scipy.interpolate import CubicSpline
np.set_printoptions(threshold=np.inf)
from prettytable import PrettyTable  
import matplotlib as mpl
import matplotlib.dates as mdates

c_dir =os.getcwd()
path = '/Figures/'
out = c_dir +path
if not os.path.exists(out):
    os.makedirs(out)

np.random.seed(12345)
tf.set_random_seed(12345)
#tf.compat.v1.disable_eager_execution()
class SIRD:
    def __init__(self, I, R, S,D, T,N0, layers):
        self.t = T
        self.I = I
        self.R = R
        self.S = S
        self.D = D
        self.layers = layers
        self.N = N0
        self.lb = T.min()
        self.ub = T.max()
        # self.gamma = tf.Variable([1], constraint=lambda x: tf.abs(x),dtype=tf.float32)
        # Initialize NN
        self.weights1, self.biases1 = self.initialize_NN(self.layers)
        self.weights2, self.biases2 = self.initialize_NN(self.layers)
        self.weights3, self.biases3 = self.initialize_NN(self.layers)
        self.weights4, self.biases4 = self.initialize_NN(self.layers)
        self.weights5, self.biases5 = self.initialize_NN(self.layers)
        self.weights6, self.biases6 = self.initialize_NN(self.layers)
        self.weights7, self.biases7 = self.initialize_NN(self.layers)
        self.weights8, self.biases8 = self.initialize_NN(self.layers)
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.I_tf = tf.placeholder(tf.float32, shape=[None, self.I.shape[1]])
        self.R_tf = tf.placeholder(tf.float32, shape=[None, self.R.shape[1]])
        self.S_tf = tf.placeholder(tf.float32, shape=[None, self.S.shape[1]])
        self.D_tf = tf.placeholder(tf.float32, shape=[None, self.D.shape[1]])
        
        #lossess
        self.total_loss =[]
        self.loss_data =[]
        self.loss_phys =[]
        self.total_time=[]
        
        self.S_pred, self.I_pred, self.R_pred,self.D_pred =self.net_sird(self.t_tf)
        
        self.beta_pred,self.gamma_pred, self.delta_pred, self.compliance_pred =self.net_param(self.t_tf)
        
        self.e1, self.e2, self.e3, self.e4, self.e5 = self.net_residual(self.t_tf)
        
        self.lossPhy = tf.reduce_mean(tf.abs(self.e1)) + tf.reduce_mean(tf.abs(self.e2)) +\
                        tf.reduce_mean(tf.abs(self.e3)) + tf.reduce_mean(tf.abs(self.e4)) +\
                        tf.reduce_mean(tf.abs(self.e5))
        self.S_loss = 0
        self.I_loss = 0
        self.R_loss = 0
        self.D_loss = 0
        iter = 0
        for i in range(len(T)):
            if T[i]%1 == 0:
                self.S_loss += tf.abs(self.S_tf[iter] - self.S_pred[i])
                self.I_loss += tf.abs(self.I_tf[iter] - self.I_pred[i])
                self.R_loss += tf.abs(self.R_tf[iter] - self.R_pred[i])
                self.D_loss += tf.abs(self.D_tf[iter] - self.D_pred[i])
                iter += 1
        self.lossData = (self.I_loss + self.R_loss + self.S_loss+self.D_loss)/iter
        self.loss = self.lossData + self.lossPhy
        '''
        self.loss = tf.reduce_mean(tf.abs(self.I_tf - self.I_pred)) + tf.reduce_mean(tf.abs(self.R_pred - self.R_tf)) +\
                    tf.reduce_mean(tf.abs(self.S_tf - self.S_pred)) +\
                    tf.reduce_mean(tf.abs(self.E1_pred))+ tf.reduce_mean(tf.abs(self.E2_pred)) +\
                    tf.reduce_mean(tf.abs(self.E3_pred)) + tf.reduce_mean(tf.abs(self.E4_pred)) 
        '''
        
        self.optimizer_Adam = tf.compat.v1.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        # Optimizers
        '''
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})    
        '''                                                                                          
        init = tf.global_variables_initializer()
        self.sess.run(init)
         
        
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers)
        
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.nn.softplus(tf.add(tf.matmul(H, W), b))
        return Y

    def neural_net1(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.nn.sigmoid(tf.add(tf.matmul(H, W), b))
        return Y
    
    def net_sird(self, t):
        NN=self.N
        S = self.neural_net1(t, self.weights1, self.biases1)
        I = self.neural_net1(t, self.weights2, self.biases2)
        R = self.neural_net1(t, self.weights3, self.biases3)
        D = self.neural_net1(t, self.weights4, self.biases4)
        return S, I, R, D
    
    def net_param(self, t):
        NN=self.N
        beta= self.neural_net1(t, self.weights5, self.biases5)
        gamma = self.neural_net1(t,self.weights6,self.biases6)
        delta = self.neural_net1(t,self.weights7,self.biases7)
        compliance = self.neural_net1(t,self.weights8,self.biases8)
        return beta, gamma, delta, compliance
        

    def net_residual(self, t):
        NN=self.N  
        S, I, R, D =self.net_sird(t)
        # print(S.shape)
        beta, gamma, delta, compliance =self.net_param(t) 
        St = tf.gradients(S, t)[0]
        It = tf.gradients(I, t)[0]  
        Rt = tf.gradients(R, t)[0]
        Dt = tf.gradients(D, t)[0]
        
        # print(St.shape)
        
        e1 = St + beta*(1.0-compliance)*S*I
        e2 = It - beta*(1.0-compliance)*S*I + gamma*I +delta*I
        e3 = Rt - gamma*I
        e4 = Dt - delta*I
        e5 = 1.0 - (S+I+R+D)
        return e1, e2, e3, e4, e5

    def callback(self, loss):
        print('Loss: %.3e' % (loss))
    def train(self, nIter):
        tf_dict = {self.t_tf: self.t, self.I_tf: self.I,self.R_tf: self.R,self.S_tf: self.S, self.D_tf: self.D}
                   
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            loss_t = self.sess.run(self.loss, tf_dict)
            loss_d = self.sess.run(self.lossData, tf_dict)
            loss_p = self.sess.run(self.lossPhy, tf_dict)
            self.total_loss.append(loss_t)
            self.loss_data.append(loss_d)
            self.loss_phys.append(loss_p)
            elapsed = time.time() - start_time
            self.total_time.append(elapsed)
            # Print
            if it % 500 == 0:
                elapsed = time.time() - start_time
                loss_t = self.sess.run(self.loss, tf_dict)
                # loss_d = self.sess.run(self.lossData, tf_dict)
                # loss_p = self.sess.run(self.lossPhy, tf_dict)
                print('It: %d, Loss: %.3e, Time: %.2f' % 
                      (it, loss_t,elapsed))
                start_time = time.time()
        '''
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss],
                                loss_callback = self.callback)
        '''              
    def predict(self, t_star):
        tf_dict = {self.t_tf: t_star}
        S = self.sess.run(self.S_pred, tf_dict)
        I = self.sess.run(self.I_pred, tf_dict)
        R = self.sess.run(self.R_pred, tf_dict)
        D = self.sess.run(self.D_pred, tf_dict)
        beta = self.sess.run(self.beta_pred,tf_dict)
        gamma = self.sess.run(self.gamma_pred,tf_dict)
        delta = self.sess.run(self.delta_pred,tf_dict)
        compliance = self.sess.run(self.compliance_pred,tf_dict)
        return S,I,R,D, beta, gamma, delta, compliance
    
##prepare data

data =pd.read_csv("tndata.csv")
def data_preprocess(data, lb, ub, N0):
    tdat=data.reindex(index=data.index[::-1])
    ic = tdat["TOTAL_CASES"]
    dc = tdat["TOTAL_DEATHS"]
    re = tdat["TOTAL_INACTIVE_RECOVERED"]
    y1, y2 =np.array(ic.values).reshape((-1,1)), np.array(dc.values).reshape((-1,1))
    y3   =np.array(re.values).reshape((-1,1))
    y4 =y1-y3-y2 #infected cases
    I, R, D =y1[lb:ub,:],y3[lb:ub,:], y2[lb:ub,:]
    S =N0-I-R-D
    length =int(ub-lb)
    T = np.arange(0,length).reshape(length,1)
    return S, I, R, D, T

N0 = 6.82*1.e6
lb =19 #change this
ub =280
length=int(ub-lb)
# eps =[1e-1, 1e-2, 1e-3, 1e-5, 1e-7, 1e-9]
SS, II, RR, DD, T=data_preprocess(data, lb, ub, N0)
S_original=SS/N0
I_original=II/N0
R_original=RR/N0
D_original=DD/N0

l=4
neuron =[32,64]
T = np.arange(0,length - 0.05,0.05)#here I used Nt with 0.1 stepsize
T = T.reshape(len(T),1)
for i in range(2):
    layers =[1] +l*[neuron[i]] +[1]
    model = SIRD(II/N0,RR/N0,SS/N0,DD/N0, T,1,layers)
    model.train(40000)
    #predict output
    TT = np.arange(0,length).reshape(length,1)
    s_p,i_p,r_p,d_p,beta_p, gamma_p, delta_p, compliance_p= model.predict(TT)
    total_time =sum(model.total_time)
    ##Get errors
    I_or =I_original.reshape(-1,)
    I_p  =i_p.reshape(-1,)
    test_actual=I_or*N0
    test_pred =I_p*N0
    rmse =np.sqrt(mean_squared_error(test_actual, test_pred))
    mape =np.linalg.norm((test_pred-test_actual),2)/np.linalg.norm(test_actual, 2)
    ev =1- (np.var(test_pred-test_actual)/np.var(test_actual))
    rel =np.sum((test_actual-test_pred)**2/(test_actual**2))
    #Error Metrics
    with open(out+'cpupinn_{}_{}.txt'.format(l,neuron[i]), 'w') as f:
        print("Error metrics for  layers ={} with fixed neuron ={}".format(l,neuron[i]), file=f)
        print("=======================================================\n", file=f)
        print('Total CPU Time',total_time , file=f)
        print('RMSE',rmse, file=f)
        print('MAPE',mape, file=f)
        print('EV', ev, file=f)
        print('REL', rel, file=f)
        print("=======================================================\n", file=f)

This code sets up a physics-informed neural network (PINN) to estimate parameters in an SIR model, which is a mathematical model to understand the dynamics of infectious diseases. 

The SIR model is an acronym for 'Susceptible', 'Infected', and 'Recovered', which are the three compartments the model considers. It relies on a set of differential equations that relate these compartments to each other, and uses two key parameters - beta (the infection rate) and gamma (the recovery rate). 

The neural network structure is defined in the class `sir_param`. You have a number of attributes for the class such as the initial conditions of the disease (`s0`, `i0`, `r0`), the size of the population (`N`), the infection and recovery rates (`beta` and `gamma`), the learning rate (`v`), and the data split ratio (`split`). 

Then, it defines various methods, such as `xzavier` for Xavier initialization of the weights, `w_b` to initialize weights and biases, `network` and `sir_network` to define the neural network, and `residual_sir` to calculate the residuals of the SIR model. 

The key to the PINN approach here is in the `residual_sir` method, where the network predictions are plugged into the SIR equations. The residuals are then minimized during the training process to ensure that the network solutions are consistent with the underlying SIR model.

Finally, it defines a `train` method for training the model and a `predict` method to get the predictions from the model.

However, the code seems to use tensorflow version 1 (`tensorflow.compat.v1`) where you have disabled version 2 behaviour at the beginning (`tff.disable_v2_behavior()`). Starting from TensorFlow 2.0, eager execution is enabled by default, which allows a more interactive frontend to TensorFlow. You might want to refactor your code to use the newer and more streamlined APIs in TensorFlow 2.x, which are generally easier to work with. 

Furthermore, for better readability and organization, consider separating the network creation, loss function definition, and training steps into different methods or even different classes.

In [1]:
#EINN_model.py

### Using PINN for parameter Estimates
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tff
tff.disable_v2_behavior()
import time
start_time = time.time()
tff.set_random_seed(1234)
class sir_param:
    def __init__(self, n_layers, i,t, lb, ub,bi,gi, v, eta, tf, U0, N, split):
        ##Initialize 
        self.i, self.t, self.v, self.eta =i, t,v, eta
        self.lb, self.ub, self.N =lb, ub, N
        self.n_layers =n_layers
        self.tf =tf
        self.split =split
        indx =int(self.split*len(i))
        ##Training 
        self.s0, self.i0, self.r0 =U0[0], U0[1], U0[2]
        self.i_train =self.i[:indx]
        self.t_train =self.t[:indx]
        ##Testing
        self.i_test =self.i[indx:]
        self.t_test =self.t[indx:]
        self.i0t =self.i_test[0:1,:]
        self.r0t =np.array([[0.0]])
        self.s0t =self.N-self.i0t- self.r0t
        self.s0t =self.s0t.reshape((-1,1))
        self.weights, self.biases =self.w_b(self.n_layers)
        self.beta =tff.Variable([bi], dtype=tff.float32)
        self.gamma =tff.Variable([gi], dtype=tff.float32)
        ##Get placeholders as containers for input and output
        self.sess =tff.Session(config=tff.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        self.t_t =tff.placeholder(tff.float32, [None, 1])
        self.i_t =tff.placeholder(tff.float32, [None, 1])
        self.s0_t =tff.placeholder(tff.float32, [None, 1])
        self.i0_t =tff.placeholder(tff.float32, [None, 1])
        self.r0_t =tff.placeholder(tff.float32, [None, 1])
        ##Placeholders and data containers
        self.tf_t =tff.placeholder(tff.float32, [None, 1])
        self.s_pred, self.i_pred, self.r_pred=self.sir_network(self.t_t)
#         self.beta_pred = self.nn_beta(self.t_t)
        self.s0_pred =self.s_pred[0]
        self.i0_pred =self.i_pred[0]
        self.r0_pred =self.r_pred[0]
        self.e1,self.e2, self.e3=self.residual_sir(self.tf_t)
         # Loss: Initial Data
        self.lossUU0 = tff.reduce_mean(tff.square(self.s0_t - self.s0_pred)) + \
            tff.reduce_mean(tff.square(self.i0_t - self.i0_pred)) + \
            tff.reduce_mean(tff.square(self.r0_t - self.r0_pred)) 
        ##Data
        self.lossD = tff.reduce_mean(tff.square(self.i_t-self.i_pred))                   
        ##Residual
        self.lossR= tff.reduce_mean(tff.square(self.e1-0.0))+\
                    tff.reduce_mean(tff.square(self.e2-0.0))+\
                    tff.reduce_mean(tff.square(self.e3-0.0))
                    
               
        
        self.loss =self.lossUU0+self.lossD+self.lossR
        self.opt =tff.train.AdamOptimizer().minimize(self.loss)
    
        init =tff.global_variables_initializer()
        self.sess.run(init)                  
        ##Initialize weights and biases
    def xzavier(self,dim):
        d1,d2 =dim[0], dim[1]
        std =np.sqrt(2.0/(d1+d2))
        return tff.Variable(tff.truncated_normal([d1,d2], stddev=std), dtype=tff.float32)
    ##Apply to all wights
    def w_b(self, n_layers):
        l=n_layers
        weights =[self.xzavier([l[j], l[j+1]]) for j in range(0, len(l)-1)]
        biases =[tff.Variable(tff.zeros([1, l[j+1]], dtype =tff.float32),dtype=tff.float32) for j in range(0,len(l)-1)]
        return weights,biases
   
    #Define the neural network
    def network(self,t, weights, biases):
        M=len(weights)+1
        z=2.0*(t-self.lb)/(self.ub-self.lb)-1.0
        for i in range(0, M-2):
             z =tff.nn.tanh(tff.matmul(z, weights[i])+biases[i])
        y_pred =tff.nn.softplus(tff.matmul(z, weights[-1])+biases[-1])
        return y_pred
    
    def sir_network(self,t):
        out =self.network(t, self.weights, self.biases)
        s, i, r =out[:,0:1], out[:,1:2], out[:,2:3]
        return s, i, r

    
    def residual_sir(self,t):
        beta, gamma, v, eta =self.beta, self.gamma, self.v, self.eta
        s, i,r =self.sir_network(t)
        s_t =tff.gradients(s, t, unconnected_gradients='zero')[0]
        i_t =tff.gradients(i, t, unconnected_gradients='zero')[0]
        r_t =tff.gradients(r, t, unconnected_gradients='zero')[0]
        N=self.N
        e1 =s_t +(beta*s*i)/N +v*eta*s
        e2 =i_t -(beta*s*i)/N +gamma*i
        e3 =r_t -gamma*i-v*eta*s
        return e1, e2, e3
    def callbacks(self, loss, beta, gamma):
        print('Loss: {}, beta: {}, gamma: {}'.format(loss,beta,gamma))   
    def train(self, epochs):
        train_dic ={self.t_t: self.t_train, self.i_t:self.i_train,  self.tf_t:self.tf,
                   self.s0_t:self.s0, self.i0_t:self.i0, self.r0_t:self.r0}
        test_dic ={self.t_t: self.t_test, self.i_t:self.i_test,  self.tf_t:self.tf,
                   self.s0_t:self.s0t, self.i0_t:self.i0t, self.r0_t:self.r0t}
        start_time = time.time()
        for i in range(epochs+1):
            self.sess.run(self.opt, train_dic)
            self.sess.run(self.opt, test_dic)
            if i%100==0:
                elapsed = time.time() - start_time
                loss_v= self.sess.run(self.loss, train_dic)
                loss_v1= self.sess.run(self.loss, test_dic)
                beta_v=self.sess.run(self.beta)
                gamma_v=self.sess.run(self.gamma)
                print('Epoch: %d, Train Loss:%.3e, Test Loss:%.3e, beta: %.2f, gamma: %.2f, Time: %.2f'%(i, loss_v, loss_v1, beta_v, gamma_v,elapsed))
                start_time = time.time()
    def predict(self,t_hat):
        tr_dic={self.t_t: t_hat}
        s_hat =self.sess.run(self.s_pred, tr_dic)
        i_hat =self.sess.run(self.i_pred, tr_dic)
        r_hat =self.sess.run(self.r_pred, tr_dic)
        return s_hat, i_hat, r_hat 

<class 'ModuleNotFoundError'>: No module named 'tensorflow'